# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.hyperdrive import BayesianParameterSampling
from azureml.train.hyperdrive import uniform, choice

In [4]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

In [5]:
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import ScriptRunConfig

In [6]:
from azureml.train.hyperdrive import (
    PrimaryMetricGoal,
    choice,
    uniform,
    loguniform
)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [7]:
ws = Workspace.from_config()
experiment_name = 'CapstoneHyperDrive'

experiment=Experiment(ws, experiment_name)

In [8]:
# NOTE: update the key to match the dataset name
found = False
key = "loandataset"
description_text = "Loan Dataset for Capstone Project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 




df = dataset.to_pandas_dataframe()
df.describe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status
count,45000.000000,4.500000e+04,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000
mean,27.764178,8.031905e+04,5.410333,9583.157556,11.006606,0.139725,5.867489,632.608756,0.222222
std,6.045108,8.042250e+04,6.063532,6314.886691,2.978808,0.087212,3.879702,50.435865,0.415744
min,20.000000,8.000000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,390.000000,0.000000
25%,24.000000,4.720400e+04,1.000000,5000.000000,8.590000,0.070000,3.000000,601.000000,0.000000
50%,26.000000,6.704800e+04,4.000000,8000.000000,11.010000,0.120000,4.000000,640.000000,0.000000
75%,30.000000,9.578925e+04,8.000000,12237.250000,12.990000,0.190000,8.000000,670.000000,0.000000
max,144.000000,7.200766e+06,125.000000,35000.000000,20.000000,0.660000,30.000000,850.000000,1.000000


In [9]:
# create director to store job files

import os

train_src_dir = "./scripts"
os.makedirs(train_src_dir, exist_ok=True)

In [10]:
!pip install xgboost

In [65]:
%%writefile {train_src_dir}/train.py

import argparse
import os
import numpy as np
from sklearn.metrics import accuracy_score
import joblib
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core.run import Run
import xgboost as xgb
from xgboost import XGBClassifier
from azureml.core import Run, Dataset, Workspace
from sklearn.preprocessing import StandardScaler

# Save model for current iteration
run = Run.get_context()

def main():
    # Add arguments to script
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str)
    parser.add_argument('--learning_rate', type=float, default=0.1, help="Step size shrinkage to prevent overfitting")
    parser.add_argument('--n_estimators', type=int, default=100, help="Number of boosting rounds")
    parser.add_argument('--max_depth', type=int, default=3, help="Maximum depth of a tree")
    parser.add_argument('--random_state', type=int, default=1, help="Random seed for reproducibility")

    args = parser.parse_args()

    run.log("Learning Rate:", np.float(args.learning_rate))
    run.log("Number of Estimators:", np.int(args.n_estimators))
    run.log("Max Depth:", np.int(args.max_depth))
    run.log("Random State:", np.int(args.random_state))

    workspace = run.experiment.workspace  # Get the workspace
    dataset_name = args.data # Get dataset name from arguments

    # Get the Tabular Dataset and convert to DataFrame
    tabular_dataset = Dataset.get_by_name(workspace, name=dataset_name)
    x_df = tabular_dataset.to_pandas_dataframe()

    y_df = x_df.pop("loan_status")

    # Identify numeric and categorical columns
    numeric_cols = x_df.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = x_df.select_dtypes(include=['object', 'category']).columns.tolist()

# Scale numeric columns
    if numeric_cols:  # Check if there are any numeric columns
        scaler = StandardScaler()
        x_df[numeric_cols] = scaler.fit_transform(x_df[numeric_cols])
  
    # One-hot encode categorical columns
    if categorical_cols: #check if there are categorical columns
        x_df = pd.get_dummies(x_df, columns=categorical_cols, drop_first=True)
  


    # Split data into train and test sets.
    x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.3, random_state=args.random_state)

    model = xgb.XGBClassifier(
        learning_rate=args.learning_rate,
        n_estimators=args.n_estimators,
        max_depth=args.max_depth,
        random_state=args.random_state
    ).fit(x_train, y_train)

    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    run.log("Accuracy", np.float(accuracy))

    # Save model for current iteration, also include the hyperparameters in filename
    os.makedirs('outputs', exist_ok=True)
    joblib.dump(model, f'outputs/xgboost_lr_{args.learning_rate}_nestimators_{args.n_estimators}_depth_{args.max_depth}_rs_{args.random_state}.joblib')

if __name__ == '__main__':
    main()

Overwriting ./scripts/train.py


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [52]:
from azureml.train.hyperdrive import BayesianParameterSampling
from azureml.train.hyperdrive import uniform, choice


In [66]:
param_sampling = RandomParameterSampling(
        {
            '--learning_rate': loguniform(-3, -1),  # Example of log uniform distribution
            '--n_estimators': choice(50, 100, 150, 200),
            '--max_depth': choice(3, 4, 5, 6),
            
        }
    )

# Specify the primary metric to optimize.
primary_metric_name = 'Accuracy'  # The name should match what your training script logs.
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE

In [67]:
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)


In [69]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create an Azure ML environment
myenv = Environment(name='my-xgboost-env')
# Create a conda environment specification
conda_dep = CondaDependencies()
conda_dep.add_conda_package('scikit-learn')
conda_dep.add_conda_package('pandas')
conda_dep.add_conda_package('xgboost')
myenv.python.conda_dependencies = conda_dep

In [70]:
# Define the training environment
script_config = ScriptRunConfig(
        source_directory=train_src_dir,  # The folder containing your training script
        script='train.py',  # The name of your training script
        compute_target="project2",
        environment=myenv,
        arguments=[
            '--data',
            'loandataset',  #  dataset name
        ],
    )
    # Configure HyperDrive
hyperdrive_config = HyperDriveConfig(
        policy=early_termination_policy,
        hyperparameter_sampling=param_sampling,
        primary_metric_name=primary_metric_name,
        primary_metric_goal=primary_metric_goal,
        run_config=script_config,
        max_total_runs=20,  # Maximum number of hyperparameter combinations to try
        max_concurrent_runs=2,  # Adjust based on your compute target's capabilities
    )

In [71]:
#TODO: Submit your experiment
hyperDrive_run = experiment.submit(hyperdrive_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hyperDrive_run).show()

In [38]:
!pip show scikit-learn


Name: scikit-learn
Version: 1.5.1
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: azureml-automl-runtime, azureml-train-automl-runtime, azureml-training-tabular, dice-ml, econml, erroranalysis, fairlearn, interpret-core, interpret_community, ml_wrappers, pmdarima, raiutils, raiwidgets, responsibleai, seqeval, shap, skl2onnx, sklearn-pandas


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [80]:
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('Best Run Metrics:', best_run_metrics)


Best Run Id:  HD_58087a3f-96de-4ff8-a6fb-22f234f55311_9
Best Run Metrics: {'Learning Rate:': 0.20616888331427718, 'Max Depth:': 6, 'Number of Estimators:': 150, 'Random State:': 1, 'Accuracy': 0.9317037037037037}


In [78]:
best_run_metrics

{'Learning Rate:': 0.20616888331427718,
 'Max Depth:': 6,
 'Number of Estimators:': 150,
 'Random State:': 1,
 'Accuracy': 0.9317037037037037}

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [82]:
# Register the model to deploy
hyperdrive_model = best_run.register_model(model_name = "hyperdrive_capstone_bestmodel", model_path = 'outputs/')
print(hyperdrive_model)

Model(workspace=Workspace.create(name='mlcloud', subscription_id='d990bb6c-7849-4109-9dd7-6cafa051c8ae', resource_group='mlcloud'), name=hyperdrive_capstone_bestmodel, id=hyperdrive_capstone_bestmodel:1, version=1, tags={}, properties={})


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

